In [76]:
import numpy as np
import pandas as pd
import data_als
import data_prepro

In [77]:
import torch
import torch.nn as nn
from torch.autograd import Variable 

import torch.optim as optim

In [78]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv',trainYearRange1=2019,trainYearRange2=2020)

In [79]:
train = data_prepro.outlierDataToNan(train,low=True,high=False,thres=10.0)
# train = data_prepro.fillZero(train)
train = data_prepro.fillnaBehind(train)
train = data_prepro.fillprevValue(train)


Int64Index([17558, 21595, 25626], dtype='int64')
index                  0
datetime               0
구미 혁신도시배수지 유출유량 적산차    2
year                   0
month                  0
dtype: int64


In [80]:
x_train,y_train = data_prepro.XDataToXAndYSeq(train)

In [81]:
x_test,y_test = data_prepro.XDataToXAndYSeq(test)

In [82]:
x_train.shape, y_train.shape

((17519, 1, 24), (17519, 1))

In [83]:
x_train = x_train.reshape(-1,24,1)

In [84]:
y_train = y_train.reshape(-1,1)

In [85]:
y_train.shape

(17519, 1)

In [86]:
x_test = x_test.reshape(-1,24,1)

In [87]:
y_test = y_test.reshape(-1,1)

In [88]:
x_train

array([[[114.],
        [ 83.],
        [ 68.],
        ...,
        [431.],
        [610.],
        [417.]],

       [[ 83.],
        [ 68.],
        [ 90.],
        ...,
        [610.],
        [417.],
        [224.]],

       [[ 68.],
        [ 90.],
        [112.],
        ...,
        [417.],
        [224.],
        [116.]],

       ...,

       [[305.],
        [362.],
        [207.],
        ...,
        [473.],
        [305.],
        [328.]],

       [[362.],
        [207.],
        [133.],
        ...,
        [305.],
        [328.],
        [347.]],

       [[207.],
        [133.],
        [153.],
        ...,
        [328.],
        [347.],
        [335.]]])

In [89]:
y_train

array([[116.],
       [ 49.],
       [ 31.],
       ...,
       [335.],
       [141.],
       [112.]])

In [90]:
y_test

array([[ 84.],
       [ 87.],
       [ 84.],
       ...,
       [396.],
       [350.],
       [197.]])

In [91]:
X_train_tensors = Variable(torch.Tensor(x_train))
X_test_tensors = Variable(torch.Tensor(x_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

X_train_tensors_final = torch.reshape(X_train_tensors,   X_train_tensors.shape)
X_test_tensors_final = torch.reshape(X_test_tensors,  X_test_tensors.shape) 

print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([17519, 24, 1]) torch.Size([17519, 1])
Testing Shape torch.Size([8399, 24, 1]) torch.Size([8399, 1])


In [92]:
try:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # device
except:
    device = "cpu"
print(torch.cuda.get_device_name(0))

Tesla M40 24GB


In [93]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [94]:
from torch.nn.utils import weight_norm

m = weight_norm(nn.Linear(20, 40), name='weight')
m # Linear(in_features=20, out_features=40, bias=True)
m.weight_g.size() # torch.Size([40, 1])
m.weight_v.size() # torch.Size([40, 20])

torch.Size([40, 20])

In [95]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [96]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [97]:
import torch.nn.functional as F
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        o = self.linear(y1[:, :, -1])
        return o

In [98]:
num_epochs = 30000 #1000 epochs
learning_rate = 0.01 #0.001 lr

kernel_size = 5
stride = 1
dilation = 1
padding=2
dropout=0.2
input_size = 24 #number of features
hidden_size = 256 #number of features in hidden state
num_channels = [512,512,256] #number of stacked lstm layers

output_size = 1 #number of output classes 

In [99]:
tcn = TCN(input_size, output_size, num_channels, kernel_size, dropout).to(device)

loss_function = torch.nn.L1Loss()    # mean-squared error for regression
optimizer = torch.optim.Adam(tcn.parameters(), lr=learning_rate)  # adam optimizer

In [100]:
for epoch in range(num_epochs):
    outputs = tcn.forward(X_train_tensors_final.to(device)) #forward pass
    
    # obtain the loss function
    loss = loss_function(outputs, y_train_tensors.to(device))

    if epoch%20 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()),end=', ')
        outputs = tcn.forward(X_test_tensors_final.to(device)) #forward pass
        test_loss = loss_function(outputs, y_test_tensors.to(device))
        # loss.backward() #calculates the loss of the loss function

        print("test data loss",test_loss.item())

    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    loss.backward() #calculates the loss of the loss function
    optimizer.step()

Epoch: 0, loss: 250.14949, test data loss 259.26007080078125
Epoch: 20, loss: 250.83366, test data loss 259.98541259765625
Epoch: 40, loss: 225.11330, test data loss 232.98731994628906
Epoch: 60, loss: 141.45525, test data loss 143.91183471679688
Epoch: 80, loss: 139.98494, test data loss 141.7073516845703
Epoch: 100, loss: 140.06993, test data loss 141.88394165039062
Epoch: 120, loss: 139.99059, test data loss 141.3670654296875
Epoch: 140, loss: 140.14348, test data loss 141.63641357421875
Epoch: 160, loss: 139.58258, test data loss 141.60183715820312
Epoch: 180, loss: 139.77516, test data loss 141.1579132080078
Epoch: 200, loss: 139.39783, test data loss 141.2375030517578
Epoch: 220, loss: 139.25903, test data loss 140.7466278076172
Epoch: 240, loss: 139.26355, test data loss 140.48175048828125
Epoch: 260, loss: 139.00117, test data loss 140.15478515625
